<a href="https://colab.research.google.com/github/JoseAguirre-10/Tarea-2-Opti.-Mat./blob/main/P2_Tarea_2_Opti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 22.5 MB/s eta 0:00:00


In [2]:
import numpy as np
from gurobipy import*

In [3]:
#mejor utilizamos librerias para resolver el problema xd:

betha = {
    0:0,
    1:100,
    2:120,
    3:130,
    4:140,
    5:150,
    6:200,
    7:0
} #betha es un vector columna


c = {
    0:{1:10, 2:25, 3:30, 6:20},
    1:{4:15, 5:15},
    2:{5:15, 6:0},
    3:{6:10},
    4:{7:25},
    5:{7:80},
    6:{7:25},
    7:{}
}  # c es una matriz de 8x8


r = {
    0: {1: 500, 2: 350, 3: 500, 6: 600},
    1: {4: 250, 5: 350},
    2: {5: 350},
    3: {6: 500},
    4: {7: 800},
    5: {7: 500},
    6: {7: 500},
    7: {}
}  # r es una matriz de 8x8

n = 8


m = Model('Penalización')

#defino la varaible


x = m.addVars(n,n, vtype= GRB.CONTINUOUS,lb= 0, name = 'elemtos_por_bodega')

#añado las restricciones:


#hay que bloquear aquellos caminos donde r[i][j] == 0
for i in range(n):
  for j in range(n):
    if j not in r[i]:
      m.addConstr(x[i,j] == 0, name=f'Bloqueo_{i}_{j}')
    else:
      m.addConstr(x[i,j] <= r[i][j], name=f'Capacidad_{i}_{j}')


#Primera bodega tenga 1000

m.addConstr(quicksum(x[0, j] for j in range(n)) == 1000, name='Inicio')

#Ultima bodega tenga 1000

m.addConstr(quicksum(x[i,7] for i in range(n)) == 1000, name='Llegada')


#lo que entra por un nodo debe ser igual a lo que sale por este:

for i in range(1,7):
  entrada = quicksum(x[j,i] for j in range(n))
  salida = quicksum(x[i,j] for j in range(n))
  m.addConstr(entrada == salida, name=f'balance_{i}')

#definimos la función Objetivo:

costo_transporte = quicksum(c[i][j]*(x[i,j]*x[i,j]) for i in range(n) for j in range(n) if j in r[i])

costo_bodegaje = quicksum(betha[j]*(quicksum(x[k,j] for k in range(n))) for j in range(n) if j)

m.setObjective(costo_bodegaje + costo_transporte, GRB.MINIMIZE)

m.setParam('OutputFlag', 0)
m.optimize()

if m.status == GRB.OPTIMAL:
    print("Solución óptima encontrada:")
    for i in range(n):
        for j in range(n):
            if x[i, j].x > 1e-6:  # Evitar imprimir valores cercanos a cero
                print(f'x[{i},{j}] = {x[i, j].x:.2f}')
else:
    print("No se encontró solución óptima")




Restricted license - for non-production use only - expires 2026-11-23
Solución óptima encontrada:
x[0,1] = 365.54
x[0,2] = 134.46
x[0,3] = 165.58
x[0,6] = 334.42
x[1,4] = 250.00
x[1,5] = 115.54
x[2,5] = 134.46
x[3,6] = 165.58
x[4,7] = 250.00
x[5,7] = 250.00
x[6,7] = 500.00


In [7]:
#ahora tenemos que tratar de lograr hacer la p2. mi unico problema es que necesito tener solo igualdades como estriccion.

#DEFINICIOÓN DE CONSTANTES PARA ITERACIÓN.
mu = 1
n= 8
epsilon = 10**-5
for k in range(10):
  m2 = Model('Penalización')

  x = m2.addVars(n,n, vtype= GRB.CONTINUOUS,lb= 0, name = 'elemtos_por_bodega')

  R0 = quicksum(x[0, j] for j in range(n)) - 1000

  R1 = quicksum(x[i,7] for i in range(n)) - 1000

  R2 = []  # Balance de flujo para nodos intermedios

  for l in range(1, 7):
      R2.append(quicksum(x[i,l] for i in range(n)) - quicksum(x[l, j] for j in range(n)))

  #Esto de aca define las cotas inferiores y superiores Q en las cuales esta definido x

  for i in range(n):
      for j in range(n):
          if j in r[i]:  # Asegurar que la ruta existe
              m2.addConstr(x[i,j] <= r[i][j], f"Cota_superior_{i}_{j}")
          else:
            m2.addConstr(x[i,j] == 0, f"No_negatividad_{i}_{j}")

  #entonces ahora debemos definir phi

  costo_transporte = quicksum(c[i][j] * x[i,j] * x[i,j] for i in range(n) for j in c[i])
  costo_bodegaje = quicksum(betha[j] * quicksum(x[i,j] for i in range(n)) for j in range(n))

  f = costo_transporte + costo_bodegaje

  penalizacion = R0 * R0 + R1 * R1 + quicksum(r * r for r in R2)

  m2.setObjective(f + mu * penalizacion, GRB.MINIMIZE)

  # Resolver
  m2.setParam('OutputFlag', 0)
  m2.optimize()


  if m2.status == GRB.OPTIMAL:
      #imprimir el resultado que obtuvimos y evaluarlo
      R2_value = [r.getValue() for r in R2]
      valor_penalizacion = (R0.getValue()**2 + R1.getValue()**2 + sum(r**2 for r in R2_value))
      print(f"Iteración {k+1}: mu={mu}, penalización={mu*valor_penalizacion:.2f}, f(x) = {f.getValue():.2f}, Phi(x) = {f.getValue()+mu*valor_penalizacion:.2f}")
      print('El resultado es el siguiente: ')
      #debemos empezar agregar las condiciones para seguir o no seguir iterando
      if valor_penalizacion < epsilon or k ==9: #se deja de iterar y se imprime el resutlado
        print("Solución óptima encontrada:")

      for i in range(n):
          for j in range(n):
              if x[i, j].x > 1e-6:  # Evitar imprimir valores cercanos a cero
                  print(f'x[{i},{j}] = {x[i, j].x:.2f}')

      if valor_penalizacion	>= epsilon or k!=9:
        mu= 10*mu
  else:
    print("No se encontró solución óptima")








Iteración 1: mu=1, penalización=1547815.68, f(x) = 216375.07, Phi(x) = 1764190.74
El resultado es el siguiente: 
x[0,1] = 71.74
x[0,2] = 29.87
x[0,3] = 24.89
x[0,6] = 36.83
x[1,4] = 2.16
x[1,5] = 0.35
x[4,7] = 35.34
x[5,7] = 11.32
x[6,7] = 36.67
Iteración 2: mu=10, penalización=4378121.07, f(x) = 4611713.15, Phi(x) = 8989834.22
El resultado es el siguiente: 
x[0,1] = 256.88
x[0,2] = 111.02
x[0,3] = 99.08
x[0,6] = 173.51
x[1,4] = 109.75
x[1,5] = 49.49
x[2,5] = 35.05
x[3,6] = 43.29
x[4,7] = 183.73
x[5,7] = 68.65
x[6,7] = 214.32
Iteración 3: mu=100, penalización=1967884.66, f(x) = 14765241.81, Phi(x) = 16733126.46
El resultado es el siguiente: 
x[0,1] = 351.31
x[0,2] = 126.82
x[0,3] = 154.65
x[0,6] = 304.92
x[1,4] = 250.00
x[1,5] = 74.65
x[2,5] = 96.83
x[3,6] = 139.40
x[4,7] = 288.55
x[5,7] = 156.76
x[6,7] = 444.00
Iteración 4: mu=1000, penalización=435925.37, f(x) = 18664638.58, Phi(x) = 19100563.95
El resultado es el siguiente: 
x[0,1] = 357.96
x[0,2] = 129.85
x[0,3] = 167.46
x[0,6] = 3